In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} seaborn

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import hvplot.pandas
import hvplot.xarray
from irrigation_detection import subset_ds
import rioxarray
import panel.widgets as pnw
import datetime as dt

## Load Data (GeoDataFrame, Xarray)

In [ ]:
# Datasets
LTS = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.shp')
BDF5L = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\BFDL_ML46\BFD5L_subset.shp')
NDVI_10Dmax = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\NDVI_10Dmax_1km.nc')#.rename({'lon':'longitude', 'lat':'latitude'})
RADOLAN = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\RADOLAN_1km.nc').rename({'SF': 'precipitation'})
SSM = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\SSM_1km_V1.nc')
EVAPO_R = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\EVAPOTRANSPIRATION_REAL.nc')
EVAPO_P = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\EVAPOTRANSPIRATION_POTENTIAL.nc')
L7_sr_ndvi = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\l7_sr_ndvi_100m.nc')
L8_sr_ndvi = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\l8_sr_ndvi_100m.nc')
S2_1C = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\s2_1c.nc')
S2_1C_ndvi = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\s2_1c_ndvi_100m.nc')

## Select Subset of Data

In [ ]:
# Define a point of interest (x,y), so (longitude, latitude).
point = (8.5007, 49.9389) #LTS ID 42142960
#Define roi
aoi = gpd.read_file('C://Users/USER/Desktop/Master_Irrigation/03_GIS/gg_test_field/gg_test_field_1.geojson')
shape_aoi = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/gg_test_field/gg_test_field_1.shp'

In [ ]:
ds_collection = [x.sel(lon=point[0], lat=point[1], method='nearest').to_dataframe() for x in [NDVI_10Dmax,RADOLAN, SSM, EVAPO_P, EVAPO_R]]
#Remove time from datetime object
ds_collection[1].index = ds_collection[1].index.normalize()

## Preprocess

In [ ]:
#Initialze Dataframe with daily datetime index
gg_test_field_1 = pd.DataFrame(index =pd.DatetimeIndex(data = pd.date_range(start='1/1/2015', end='30/09/2021')))

#Join data from other sources
gg_test_field_1 = gg_test_field_1.join(ds_collection[0])[['NDVI']]
gg_test_field_1 = gg_test_field_1.join(ds_collection[1])[['NDVI', 'precipitation']]
gg_test_field_1 = gg_test_field_1.join(ds_collection[2])[['NDVI', 'precipitation', 'ssm']]
gg_test_field_1 = gg_test_field_1.join(ds_collection[3])[['NDVI', 'precipitation', 'ssm', 'evapo_p']]
gg_test_field_1 = gg_test_field_1.join(ds_collection[4])[['NDVI', 'precipitation', 'ssm', 'evapo_p', 'evapo_r']].reset_index()

In [ ]:
# Create a DataFrame from point of interest.
p_df = pd.DataFrame.from_dict({'x':[8.5007], 'y':[49.9389]})

In [ ]:
# Create GeoDataFrame from DataFrame.
p_gdf = gpd.GeoDataFrame(p_df, geometry=gpd.points_from_xy(p_df.x, p_df.y), crs='EPSG:4326')

In [ ]:
## Table data query.
# Query LTS data.
p_gdf = p_gdf.sjoin_nearest(LTS, lsuffix='LTS_left', rsuffix='LTS_right', distance_col='LTS_distance')
# Query BDF5L data.
p_gdf = p_gdf.sjoin_nearest(BDF5L, lsuffix='BDF5L_left', rsuffix='BDF5L_right', distance_col='BDF5L_distance')

In [ ]:
#export to csv
p_gdf.to_csv('gg_data.csv')

In [ ]:
## Grid data query.
test = SSM.sel(lon=point[0], lat=point[1], method='nearest')
test = test.where(test.ssm != np.nan, drop=True)
test.ssm.hvplot.line(groupby='time.year')

## Plotting

In [ ]:
# xarray plotting subplots for year and group
for year, group in gg_test_field_1.groupby(pd.Grouper(key='index',freq='1Y')):
    plot = group.interpolate(method='ffill').plot(x='index',subplots=True,sharex=True, figsize=(32,16))

In [ ]:
# Plot GeoDataFrame table.
p_gdf.hvplot.table(columns=[x for x in list(p_gdf.columns.values) if x != 'geometry'])